In [3375]:
import json
import pandas as pd
import numpy as np
from pathlib import Path 
import re
import ast

In [3376]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/"
input_directory = Path(f'{main_path}TikTok/Input_test')  
print(input_directory)
max_columns = 7


/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/structure_donations/Processed_structure_donations/TikTok/Input_test


In [3377]:
for i in range(max_columns):
        col_path = [f"col_path_{i}" for i in range(max_columns)]
        col_path_values = [f"col_path_{i}_values" for i in range(1, max_columns)]
        col_path_list = [f"col_path_{i}_lIST" for i in range(1, max_columns)]

In [3378]:
print(col_path_values)

['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values', 'col_path_6_values']


### loading_data()

In [3379]:
def loading_data(data):
    

    with open(data, 'r') as file:
        data = json.load(file)

    df = pd.DataFrame()
    df['col_path_0_values'] = data
    df['col_path_0'] = ''
    
    df.reset_index(inplace = True)
    df = df.drop('index', axis = 1)


        

    for i in range(1,max_columns):   
        df[f'col_path_{i}'] = ''
        df[f'col_path_{i}_values'] = ''
        df[f'col_path_{i}_LIST'] = ''


    
    return(df)



### flatten_json()

In [3380]:
def flatten_json(df):
    

    for i in range(max_columns):
        for ix, row in df.iterrows():
            value= row[f'col_path_{i}_values']
            
            if isinstance(value, dict):
                
                for k, v in value.items():
                    if isinstance(v, list):
                        v = v[0]
                        list_holder = 'LIST'
                    
                    
                    else:
                        list_holder = 'NO LIST'
                    

                    new_row = row.copy()
                    new_row[ f'col_path_{i+1}_values'] = v
                    new_row[ f'col_path_{i}'] = k
                    new_row[f'col_path_{i+1}_LIST'] = list_holder
                    df.loc[len(df)] = new_row
                        


            else:
                continue

    df = df.replace(r'^\s*$', np.nan, regex=True)
    df_red = df.drop(columns=[col for col in df.columns if col.endswith("_LIST")])
    
    df_red['last_valid_index'] = df_red.apply(pd.Series.last_valid_index, axis=1)
    df_red = df_red.drop(columns=[col for col in df.columns if col.endswith("values")])


    df = pd.merge(df, df_red, on = col_path, how='left')
    
    
    return(df)

### process_col_path()
The 'process_col_path()' function checks whether the value in a row of for one of the column paths is actually a datatype and stores this value in a column data_type and replaces the original value with NA. The data types are the lowest level values in the JSON files

In [3381]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object', 'str', 'int', 'float', 'bool', 'dict', 'list']



# Define the function 'process_col_path()'
def process_col_path(df, col_path_values, data_types):
    df['data_type'] = ''

    """
    row: Rows in the dataframe
    columns: List of column names of column path columns 
    data_types: List of the values that are data types
    """
    
    for ix, row in df.iterrows():
        for col in col_path_values:
            

            #If the value stored in the column is found in the list 'data_types', 
            if row[col] in data_types:
                # this value is placed in the column 'data_type'
                value = row[col]
                
                df.at[ix, 'data_type'] = value
            else:
                continue
               
    
                
    return df

### remove_intermediate_rows()

In [3382]:
def remove_intermediate_rows(df):
    drop_index = []
    for ix, row in df.iterrows():
            last_value = row[f"{row['last_valid_index']}"]
           
        
            if isinstance(last_value, str):
                    if last_value in (data_types):
                            col =  row['last_valid_index']
                            col_level = re.findall(r'\d+', col)
                            col_level = col_level[0]
                           
                            
                            df.at[ix, f"col_path_{col_level}_LIST"] = np.nan

                            
            else:
                   drop_index.append(ix)
   
    df = df.drop(drop_index)

    
    return df

### extract_path()

In [3383]:
def extract_path(df, max_columns):
    
    df['path']= ''
    
    for ix, row in df.iterrows():
        path = []
    
        for col in [f"col_path_{i}" for i in range(max_columns)]:
            val = row[col]
        
            if pd.notna(val):

                path.append(str(val).strip())

      
        
        df.at[ix, 'path'] = path
       
    
        

    return df




### list_path()

In [3384]:
def list_path(df):
    
    df['list_path'] = ''
    df['subfield_path'] = ''
    df['column_name'] = ''
    df['var_type'] = ''
    


    for ix, row in df.iterrows():
        
        path = row['path'] 
        
        
        for i in range(1, len(path)):
           
            if row[f"col_path_{i}_LIST"] == 'LIST':
                var_type = 'list'
                list_path = json.dumps(path[:-1])
                subfield_path = path[-1]
                column_name = subfield_path
                subfield_path = json.dumps(subfield_path)

            elif row[f"col_path_{i}_LIST"] == "NO LIST":
                var_type = 'static'
                list_path = np.nan
                subfield_path = json.dumps(path)
                column_name = path[-1]
             
            elif row[f"col_path_{i}_LIST"] == 'nan':
                var_type = 'skip'
                list_path = np.nan
                subfield_path = json.dumps(path)
                column_name = path[-1]
               
            else:
                var_type = 'skip'
                list_path = np.nan
                subfield_path = json.dumps(path)
                column_name = path[-1]


           
    
        df.at[ix, 'list_path'] = list_path
        df.at[ix, 'subfield_path'] = subfield_path
        df.at[ix, 'column_name'] = column_name
        df.at[ix, 'var_type'] = var_type

    
      
        
    return df

### clean_and_store()
This function orders the columns in the DataFrame, resets the index, fills the NA and saves the DataFrame as CSV

In [3385]:
def clean_and_store(df, file_name):

    """
    df: The dataframe that will be cleaned and stored
    file_name: The filename of data structure that is being processed
    """

  
   
    df=  df[df['col_path_0'] != 'Direct Message']
    
    
  
    df = df[['column_name', 'path', 'list_path', 'subfield_path', 'var_type', 'data_type']]
    # Save the DataFrame 
    df.to_csv(f"{main_path}TikTok/Output/Output_" + file_name + '.csv', index=False)

    return df

### structure_donations()
The structure_donations() function executes all functions above and results in a saved DataFrame for each data structure.

In [3386]:
def structure_donations(data, col_path, max_columns):
     # Store the path to the data structure
    data = Path(data)  
    
    # Save teh file name of the data structure
    file_name = Path(data).stem 

    df = loading_data(data)
    print('FINISH: loading_data()')
    df = flatten_json(df)
    print('FINISH: flatten_json()')
    df = process_col_path(df, col_path_values, data_types)
    print('FINISH: process_col_path()')
    df = remove_intermediate_rows(df)
    print('FINISH: remove_intermediate_rows()')
    df = extract_path(df, max_columns)
    print('FINISH: extract_path()')
    df = list_path(df)
    print('FINISH: list_path()')
    df = clean_and_store(df, file_name)
    print('FINISH: clean_and_store()')

    return(df)
    
    

    

## Execute 'structure_donations()': Transform data structures from JSON format to tabular format

In [3387]:
# Execute the 'structure_donations()' function for each file (data structure) in the input directory
for file in input_directory.iterdir():  
    if file.is_file():  
        print(file.name)
        structure_donations(file, col_path, max_columns)

HJ_tiktol_json_strcure.json
FINISH: loading_data()


/tmp/ipykernel_989656/2132457031.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
TT_structure_tiktok_takeout_ro.json
FINISH: loading_data()


/tmp/ipykernel_989656/2132457031.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)
/tmp/ipykernel_989656/2132457031.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
extra_tiktok_json_structure.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
TT_structure_tiktok_takeout_es.json
FINISH: loading_data()


/tmp/ipykernel_989656/2132457031.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)
/tmp/ipykernel_989656/2132457031.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
TT_structure_user_data_tiktok.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
TT_structure_tiktok_takeout_dutch.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()


/tmp/ipykernel_989656/2132457031.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)
/tmp/ipykernel_989656/2132457031.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
TT_json_structure_MvdV.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()
tiktok_takeout_dutch_structure.json
FINISH: loading_data()
FINISH: flatten_json()
FINISH: process_col_path()
FINISH: remove_intermediate_rows()
FINISH: extract_path()
FINISH: list_path()
FINISH: clean_and_store()


/tmp/ipykernel_989656/2132457031.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


## Merge all data structures into one schema_df

In [3388]:
# Path to the folder containing CSV files
output_path = f"{main_path}TikTok/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames
merged_df = pd.concat(dfs, axis=0, ignore_index=True)

# Drop rows that are completely identical across all columns
merged_df = merged_df.drop_duplicates()
display(merged_df)


,column_name,path,list_path,subfield_path,var_type,data_type
0,App,"['Favorite Videos', 'App']",NaN,"[""Favorite Videos"", ""App""]",static,number
1,App,"['Follower List', 'App']",NaN,"[""Follower List"", ""App""]",static,number
2,IsFastLane,"['Follower List', 'IsFastLane']",NaN,"[""Follower List"", ""IsFastLane""]",static,boolean
3,App,"['Following List', 'App']",NaN,"[""Following List"", ""App""]",static,number
4,IsFastLane,"['Following List', 'IsFastLane']",NaN,"[""Following List"", ""IsFastLane""]",static,boolean
...,...,...,...,...,...,...
909,AIGeneratedContent,"['RecentlyDeletedPosts', 'PostList', 'AIGenera...","[""RecentlyDeletedPosts"", ""PostList""]","""AIGeneratedContent""",list,string
910,Sound,"['RecentlyDeletedPosts', 'PostList', 'Sound']","[""RecentlyDeletedPosts"", ""PostList""]","""Sound""",list,string
911,Location,"['RecentlyDeletedPosts', 'PostList', 'Location']","[""RecentlyDeletedPosts"", ""PostList""]","""Location""",list,string
912,Title,"['RecentlyDeletedPosts', 'PostList', 'Title']","[""RecentlyDeletedPosts"", ""PostList""]","""Title""",list,string


### Id creation

In [ ]:
id_df = merged_df[['path', 'column_name']]
id_df = id_df.drop_duplicates()
id_df[ 'id'] = ''



for n in range(1, max_columns):
    duplicates = id_df[id_df.duplicated(subset='id', keep=False)]

    for i, row in duplicates.iterrows():
        path = row['path']
        path =  ast.literal_eval(path)
        path_zero = path[0]
        path_rest = path[-n:]  # Varies each loop to attempt uniqueness
        id_list = [path_zero] + path_rest
        new_id = ':'.join(id_list)

        id_df.at[i, 'id'] = new_id  # Properly update the DataFrame



merged_df = pd.merge(merged_df, id_df, on = ['path', 'column_name'], how = 'left')

col = merged_df.pop('id') 
merged_df.insert(0, 'id', col) 
#merged_df['id']= merged_df['id'].fillna(merged_df['variable'])


Favorite Videos
['App']
Follower List
['App']
Follower List
['IsFastLane']
Following List
['App']
Following List
['IsFastLane']
Like List
['App']
Ad Interests
['AdInterestCategories']
Block
['App']
Settings
['App']
Comments
['App']
AIMoji
['CreateDate']
Auto Fill
['PhoneNumber']
Auto Fill
['Email']
Auto Fill
['FirstName']
Auto Fill
['LastName']
Auto Fill
['Address']
Auto Fill
['ZipCode']
Auto Fill
['Unit']
Auto Fill
['City']
Auto Fill
['State']
Auto Fill
['Country']
Profile Information
['App']
Watch Live Settings
['MostRecentModificationTimeInApp']
Watch Live Settings
['MostRecentModificationTimeInWeb']
Activity Summary
['note']
Activity Summary
['videosCommentedOnSinceAccountRegistration']
Activity Summary
['videosSharedSinceAccountRegistration']
Activity Summary
['videosWatchedToTheEndSinceAccountRegistration']
Favorite Videos
['Date']
Favorite Videos
['Link']
Following List
['Date']
Following List
['UserName']
Hashtag
['HashtagName']
Hashtag
['HashtagLink']
Like List
['date']
Like L

#### ID duplicate flags

In [3390]:
merged_df = merged_df.drop_duplicates()
dup = merged_df[merged_df.duplicated('id', keep= False)]
dup['duplicate_flag'] = 'Yes'
dup = dup[['id', 'duplicate_flag']]
merged_df = pd.merge(merged_df, dup, on = 'id', how = 'left')
merged_df['duplicate_flag'] = merged_df['duplicate_flag'].fillna(value = 'No')

/tmp/ipykernel_989656/29815283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dup['duplicate_flag'] = 'Yes'


### Clean and Save

In [3391]:
merged_df['json_name'] = ''

keep_columns = ['id', 'column_name', 
                 'path', 'list_path', 'subfield_path', 
                 'var_type', 'data_type', 'json_name', 'duplicate_flag']


merged_df = merged_df[keep_columns ]

# Save the final merged DataFrame
merged_df.to_csv(f"{main_path}TikTok/Final/Merged_structures_TT.csv", index=False)